In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import re

In [ ]:
tipo_seleccionado = None
inecuaciones = []

output = widgets.Output()
termino1_input = widgets.Text(description='Término 1:', placeholder='Ingrese el primer término')
termino2_input = widgets.Text(description='Término 2:', placeholder='Ingrese el segundo término')
boton_agregar = widgets.Button(description='Agregar inecuación', button_style='success')
boton_finalizar = widgets.Button(description='Finalizar', button_style='danger')
contenedor_inputs = widgets.VBox([])

def seleccionar_tipo(b):
    global tipo_seleccionado
    tipo_seleccionado = b.description
    
    with output:
        clear_output()
        print(f"Tipo seleccionado: {tipo_seleccionado}\n")
        print("Ingrese los términos de cada inecuación y haga clic en 'Agregar inecuación'.")
        print("Para terminar, deje vacío cualquiera de los dos términos y haga clic en 'Agregar',")
        print("o simplemente haga clic en 'Finalizar'.\n")
    
    contenedor_inputs.children = [termino1_input, termino2_input, 
                                   widgets.HBox([boton_agregar, boton_finalizar]), output]

def agregar_inecuacion(b):
    global inecuaciones
    termino1 = termino1_input.value.strip()
    termino2 = termino2_input.value.strip()
    
    if termino1 == "" or termino2 == "":
        mostrar_resultado()
        return
    
    inecuacion = f"{termino1} {tipo_seleccionado} {termino2}"
    inecuaciones.append(inecuacion)
    
    with output:
        print(f"Inecuación agregada: {inecuacion}")
    
    termino1_input.value = ""
    termino2_input.value = ""

def finalizar(b):
    mostrar_resultado()

def mostrar_resultado():
    with output:
        print("\n=== Inecuaciones ingresadas ===")
        if inecuaciones:
            for i, ineq in enumerate(inecuaciones, 1):
                print(f"{i}. {ineq}")
        else:
            print("No se ingresaron inecuaciones.")

boton_mayor = widgets.Button(description='>', button_style='info')
boton_menor = widgets.Button(description='<', button_style='info')
boton_mayor_igual = widgets.Button(description='>=', button_style='info')
boton_menor_igual = widgets.Button(description='<=', button_style='info')

boton_mayor.on_click(seleccionar_tipo)
boton_menor.on_click(seleccionar_tipo)
boton_mayor_igual.on_click(seleccionar_tipo)
boton_menor_igual.on_click(seleccionar_tipo)
boton_agregar.on_click(agregar_inecuacion)
boton_finalizar.on_click(finalizar)

print("Seleccione el tipo de desigualdad haciendo clic en uno de los botones:\n")
display(widgets.HBox([boton_mayor, boton_menor, boton_mayor_igual, boton_menor_igual]))
display(contenedor_inputs)

In [ ]:
output_resolucion = widgets.Output()
funcion_obj_input = widgets.Text(description='Función objetivo:', placeholder='Ej: 35x1 + 75x2')
tipo_optimizacion = widgets.Dropdown(options=['Maximizar', 'Minimizar'], description='Tipo:')
boton_resolver = widgets.Button(description='Resolver', button_style='primary')

def parsear_expresion(expr):
    coeficientes = {}
    terminos = re.findall(r'([+-]?\s*\d*\.?\d*)\s*([xXyY]\d*)', expr)
    
    for coef, var in terminos:
        coef = coef.replace(' ', '')
        if coef == '' or coef == '+':
            coef = 1
        elif coef == '-':
            coef = -1
        else:
            coef = float(coef)
        
        var_lower = var.lower()
        if 'x' in var_lower:
            var_num = int(var[1:]) if len(var) > 1 and var[1:].isdigit() else 1
        elif 'y' in var_lower:
            var_num = 2
        else:
            var_num = 1
        
        coeficientes[var_num] = coeficientes.get(var_num, 0) + coef
    
    return coeficientes

def parsear_inecuacion(ineq):
    partes = re.split(r'\s*(<=|>=|<|>|=)\s*', ineq)
    if len(partes) < 3:
        return None, None, None
    
    lado_izq = partes[0]
    operador = partes[1]
    lado_der = partes[2]
    
    coef_izq = parsear_expresion(lado_izq)
    coef_der = parsear_expresion(lado_der) if any(c.isalpha() for c in lado_der) else {}
    
    try:
        constante = float(lado_der) if not any(c.isalpha() for c in lado_der) else 0
    except:
        constante = 0
    
    coef_final = {}
    for var in set(list(coef_izq.keys()) + list(coef_der.keys())):
        coef_final[var] = coef_izq.get(var, 0) - coef_der.get(var, 0)
    
    return coef_final, operador, constante

def resolver_programacion_lineal(b):
    with output_resolucion:
        clear_output()
        
        if not inecuaciones:
            print("No hay inecuaciones para resolver.")
            return
        
        funcion_obj = funcion_obj_input.value.strip()
        if not funcion_obj:
            print("Debe ingresar una función objetivo.")
            return
        
        print("=== RESOLUCIÓN DE PROGRAMACIÓN LINEAL ===\n")
        print(f"Función objetivo: {tipo_optimizacion.value} {funcion_obj}")
        print(f"Restricciones:")
        for i, ineq in enumerate(inecuaciones, 1):
            print(f"  {i}. {ineq}")
        print()
        
        coef_obj = parsear_expresion(funcion_obj)
        num_vars = max(coef_obj.keys()) if coef_obj else 2
        
        c = [coef_obj.get(i, 0) for i in range(1, num_vars + 1)]
        if tipo_optimizacion.value == 'Maximizar':
            c = [-x for x in c]
        
        A_ub = []
        b_ub = []
        A_eq = []
        b_eq = []
        
        for ineq in inecuaciones:
            coef, op, const = parsear_inecuacion(ineq)
            if coef is None:
                continue
            
            coef_list = [coef.get(i, 0) for i in range(1, num_vars + 1)]
            
            if op in ['<=', '<']:
                A_ub.append(coef_list)
                b_ub.append(const)
            elif op in ['>=', '>']:
                A_ub.append([-x for x in coef_list])
                b_ub.append(-const)
            elif op == '=':
                A_eq.append(coef_list)
                b_eq.append(const)
        
        bounds = [(0, None) for _ in range(num_vars)]
        
        result = linprog(c, A_ub=A_ub if A_ub else None, b_ub=b_ub if b_ub else None,
                        A_eq=A_eq if A_eq else None, b_eq=b_eq if b_eq else None,
                        bounds=bounds, method='highs')
        
        if result.success:
            valor_optimo = -result.fun if tipo_optimizacion.value == 'Maximizar' else result.fun
            print(f"Solución óptima encontrada:")
            for i, val in enumerate(result.x, 1):
                print(f"  x{i} = {val:.4f}")
            print(f"\nValor óptimo: {valor_optimo:.4f}")
            
            if num_vars == 2:
                graficar_solucion(result.x, A_ub, b_ub, c, tipo_optimizacion.value)
        else:
            print(f"No se encontró solución. Estado: {result.message}")

def graficar_solucion(punto_optimo, A_ub, b_ub, c, tipo_opt):
    fig, ax = plt.subplots(figsize=(10, 8))
    
    max_x = max(punto_optimo[0] * 1.5, 10)
    max_y = max(punto_optimo[1] * 1.5, 10)
    
    x1 = np.linspace(0, max_x, 400)
    
    colores = ['blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray']
    
    for i, (coefs, b) in enumerate(zip(A_ub, b_ub)):
        if abs(coefs[1]) > 1e-10:
            x2 = (b - coefs[0] * x1) / coefs[1]
            color = colores[i % len(colores)]
            ax.plot(x1, x2, label=f'Restricción {i+1}', linewidth=2, color=color)
        elif abs(coefs[0]) > 1e-10:
            x_const = b / coefs[0]
            ax.axvline(x=x_const, label=f'Restricción {i+1}', linewidth=2, 
                      color=colores[i % len(colores)])
    
    vertices_x = [0]
    vertices_y = [0]
    for i in range(len(A_ub)):
        for j in range(i+1, len(A_ub)):
            A = np.array([A_ub[i], A_ub[j]])
            b_vec = np.array([b_ub[i], b_ub[j]])
            try:
                punto = np.linalg.solve(A, b_vec)
                if punto[0] >= -0.01 and punto[1] >= -0.01:
                    vertices_x.append(max(0, punto[0]))
                    vertices_y.append(max(0, punto[1]))
            except:
                pass
    
    for i in range(len(A_ub)):
        if abs(A_ub[i][1]) > 1e-10:
            x_en_0 = b_ub[i] / A_ub[i][1]
            if x_en_0 >= 0:
                vertices_x.append(0)
                vertices_y.append(x_en_0)
        if abs(A_ub[i][0]) > 1e-10:
            y_en_0 = b_ub[i] / A_ub[i][0]
            if y_en_0 >= 0:
                vertices_x.append(y_en_0)
                vertices_y.append(0)
    
    if vertices_x and vertices_y:
        from scipy.spatial import ConvexHull
        try:
            puntos = np.column_stack([vertices_x, vertices_y])
            hull = ConvexHull(puntos)
            for simplex in hull.simplices:
                ax.plot(puntos[simplex, 0], puntos[simplex, 1], 'k-', alpha=0.3)
            ax.fill(puntos[hull.vertices, 0], puntos[hull.vertices, 1], 
                   alpha=0.2, color='lightblue', label='Región factible')
        except:
            pass
    
    ax.plot(punto_optimo[0], punto_optimo[1], 'r*', markersize=25, 
            label='Punto óptimo', zorder=5, markeredgecolor='darkred', markeredgewidth=2)
    ax.annotate(f'  ({punto_optimo[0]:.2f}, {punto_optimo[1]:.2f})',
                xy=punto_optimo, xytext=(punto_optimo[0]+max_x*0.05, punto_optimo[1]+max_y*0.05),
                fontsize=12, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.8),
                arrowprops=dict(arrowstyle='->', lw=2, color='red'))
    
    ax.axhline(y=0, color='k', linewidth=1)
    ax.axvline(x=0, color='k', linewidth=1)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlabel('x₁', fontsize=14, fontweight='bold')
    ax.set_ylabel('x₂', fontsize=14, fontweight='bold')
    ax.set_title(f'Programación Lineal - {tipo_opt}', fontsize=16, fontweight='bold')
    ax.legend(loc='best', fontsize=10)
    ax.set_xlim(-0.5, max_x)
    ax.set_ylim(-0.5, max_y)
    
    plt.tight_layout()
    plt.show()

boton_resolver.on_click(resolver_programacion_lineal)

print("\n" + "="*50)
print("Ingrese la función objetivo para resolver:")
display(widgets.VBox([
    widgets.HBox([funcion_obj_input, tipo_optimizacion]),
    boton_resolver,
    output_resolucion
]))